In [26]:
import pandas as pd
import requests
from io import StringIO
import numpy as np
import uuid

In [27]:
url = "http://129.194.213.75/command/core/export-rows"
params = {
    'format': 'csv',
    'project': '1827482415382'
}

response = requests.post(url, params=params)

if response.status_code == 200:
    df = pd.read_csv(StringIO(response.text), dtype ='str', low_memory=False)
else:
    print(f"Failed to load data. Status code: {response.status_code}")

In [10]:
df = df.drop(columns=['city_wd', 'city_x', 'city_y', 'wd_identifier', 'country_name', 'coordinates_artlas', 'statefr', 'stateen','countryfr','countryen', 'coordinates_wiki', 'colonne J', 'note_Thomas', 'distance_km'])

In [29]:
df.rename(columns={'new_lat': 'lat', 'new_long': 'lng'}, inplace=True)

In [30]:
df["wkt"] = "POINT(" + df["lng"] + " " + df["lat"] + ")"

In [31]:
def generate_uuid(name):
    name_str = str(name)
    return str(uuid.uuid3(uuid.NAMESPACE_DNS, name_str))

In [32]:
df['country_uuid'] = df['country_wd'].apply(generate_uuid)

In [33]:
df.head()

,id,name,lng,lat,country,country_wd,wd_identifier,city_true,wkt,country_uuid
0,5179,Sohag,31.691666666667,26.560555555556,Egypt,Q79,Q388108,NaN,POINT(31.691666666667 26.560555555556),694abb0e-ff6c-3d02-aeb9-fd586b0c69e3
1,114911,Töss,8.7,47.5,Switzerland,Q39,Q1699150,NaN,POINT(8.7 47.5),531c9931-f6e9-3564-bb80-86b94cca9507
2,114912,Gstaad,7.2833333333333,46.466666666667,Switzerland,Q39,Q37149,NaN,POINT(7.2833333333333 46.466666666667),531c9931-f6e9-3564-bb80-86b94cca9507
3,114913,Rapperswil,8.82228,47.22557,Switzerland,Q39,Q688539,NaN,POINT(8.82228 47.22557),531c9931-f6e9-3564-bb80-86b94cca9507
4,114914,Kehrsatz,7.4708333333333,46.909444444444,Switzerland,Q39,Q68274,NaN,POINT(7.4708333333333 46.909444444444),531c9931-f6e9-3564-bb80-86b94cca9507


In [34]:
df = df.replace(r"^ +| +$", r"", regex=True)

In [35]:
columns = df.columns

# Iterate through the columns
for col in columns:
    # Check if the column name contains a space
    if ' ' in col:
        # Replace spaces with underscores
        df.rename(columns={col: col.replace(' ', '_')}, inplace=True)

In [36]:
df.to_xml('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/city/city.xml', root_name="data", pretty_print=True, index=False) 